## Problem Statement 

The assignment of demands to vehicle compartments (stacks) in order to be transported from sources to targets. <br>

##### Assumptions
- Every customer / vertex represents a single request. (A single item to be transported).
    - In case 2 items need to be picked from the same location, the model considers this as 2 pickup vertices (then the distance / cost can be initialized with 0).
    
- The Demand represents the length of the item. 

#### Data 

- $G = (V, A)$
- $P = \{1, .., n\}$ $n$ pickup vertices for $n$ requests to be transported
- $D = \{n + 1, .., 2n\}$ $n$ delivery vertices for $n$ requests to be transported
- $V = \{0, 1, .., 2n\}$, where $0$ is the *depot*.
- Cost / Distance Matrix $C$, where $c_{i, \; j}$ is the cost of arc $i$ --> $j$.
- $M = \{1,.., m\}$ the set of Vehicle Compartments (stacks).
- $CompartmentCapacities = \{q_{k} |\; k \; in \; M\}$
- $Demands = \{d_{i}, .., d_{n}\, -d_{i}, .., -d_{n}\}$, where $d_{i}$ is the demand at vertex $i$ and $d_{i + n} = -d_{i}$ 

#### Decision Variables

- $x_{i,\; j}$
    - $x_{i,\; j} = 1$ if the vehicle is travelling from $i$ to $j$
    - $x_{i,\; j} = 0$ otherwise
- $y_{i,\; k}$
    - $y_{i,\; k} = 1$ if the demand at pickup vertex $i$ is loaded in stack $k$
    - $y_{i,\; k} = 0$ otherwise
- $u_{i}$ is the position of vertex $i$ in the route (order).
    - $0 \leq u_{i} \leq 2n$
    - $u_{0} = 0$ we start at the depot
- $s_{i,\; k}$ is the load of stack $k$ upon **leaving** vertex $i$
    - Domain $ : \{0, 1, .., q_{k}\}$ 

#### Constraints
- Every customer (vertex) is visited once.
    <br>
    <center> $\sum_{j \in V} x_{i, \; j} = 1$    $\; \forall i \in V$ (2)</center>
    <center> $\sum_{j \in V} x_{j, \; i} = 1$    $\; \forall i \in V$ (3)</center>
    <br>
    
- The demand of every vertex is assigned to exactly one compartment. (Demand must be fulfilled)
    <br>
    <center> $\sum_{k \in M} y_{i, \; k} = 1$    $\; \forall i \in P$ (4)</center>
    <br>

- If $j$ is visited directly after $i$ i.e. $x_{i, \; j} = 1$, then $u_{j} \geq u_{i} + 1$
    <br>
    <center> $u_{j} \geq u_{i} + 1 - 2n(1 - x_{i, \; j}) \; \forall i \in V, \; \forall j \in P \cup D$ (5)</center>
    <br>
    
- The item must be picked before it's delivered. Precedence constraint between pickups and deliveries.
    <br>
    <center> $u_{n + i} \geq u_{i} + 1 \; \forall i \in P$ (6)</center>
    <br>
    
- The state of the stack after loading and unloading.
    - The state of the assigned stack after picking is lowerbounded by the stack before picking and the picked (loaded) demand.
    
    <br>
    <center> $s_{j, \; k} \geq s_{i, \; k} + d_{j} * y_{j, \; k} - q_{k} * (1 - x_{i, \; j}) \; \forall i \in V, \; \forall j \in P, \; \forall k \in M$ (7)</center>
    <br>
    
    - The state of the assigned stack after delivering is lowerbounded by the stack before delivering and the (negative) delivered (unloaded) demand.
    
    <br>
    <center> $s_{(n + j), \; k} \geq s_{i, \; k} + d_{(n + j)} * y_{j, \; k} - q_{k} * (1 - x_{i, \; (n + j)}) \; \forall i \in V, \; \forall j \in P, \; \forall k \in M$ (8)</center>
    <br>
    
    - LIFO stacking constraint. The state of the stack after unloading is bounded by the state of the stack just after loading and the loaded / unloaded demand. (The state after unload $d{j}$ = state before loading $d{j}$ where $- d{j}$ is equivalent to $ + d_{j + n}$.
    
    <br>
    <center> $s_{(n + j), \; k} \geq s_{j, \; k} + d_{(n + j)} * y_{j, \; k} - q_{k} * (1 - y_{j, \; k}) \; \forall j \in P, \; \forall k \in M$ (9)</center>
    <br>

- We always start at the depot.
    <br><center> $u_{0} = 0$ (10)</center><br>

- Vertices are order along the route.
    <br><center> $1 \leq u_{i}  \leq 2n \; \forall i \in P \cup D$ (11)</center><br>

- We always begin with empty stacks at the depot.
    <br><center> $s_{0, \; k} = 0 \; \forall k \in M$ (12)</center><br>

- The stack capacity is always maintained for all stacks. 
    <br><center> $0 \leq s_{i, \; k}  \leq q_{k} \; \forall i \in P \cup D \; \forall k \in M$ (13)</center><br>


#### Objectice Function
- Minimize the route cost.
 <br>
     <center>$Min \sum_{i \in V} \sum_{j \in V} c_{i, \; j} * x_{i, \; j}$ (1)</center>


In [1]:
from gurobipy import *

In [2]:
import random 
random.seed(0)

#### Data 

- $G = (V, A)$
- $P = \{1, .., n\}$ $n$ pickup vertices for $n$ requests to be transported
- $D = \{n + 1, .., 2n\}$ $n$ delivery vertices for $n$ requests to be transported
- $V = \{0, 1, .., 2n\}$, where $0$ is the *depot*.
- Cost / Distance Matrix $C$, where $c_{i, \; j}$ is the cost of arc $i$ --> $j$.
- $M = \{1,.., m\}$ the set of Vehicle Compartments (stacks).
- $CompartmentCapacities = \{q_{k} |\; k \; in \; M\}$
- $Demands = \{d_{i}, .., d_{n}\, -d_{i}, .., -d_{n}\}$, where $d_{i}$ is the demand at vertex $i$ and $d_{i + n} = -d_{i}$ 

In [3]:
# Assume we have a depot and 5 items to be transported (1 + 10 vertices)
nb_items = 5
V = [i for i in range(nb_items * 2 + 1)]
P = V[1:nb_items + 1]
D = V[nb_items + 1:]
PUD = V[1:]
N = len(P) # == nb_items

# Assume an arbitrary graph
# Distance from / to depot is 0
random.seed(0)
A = [[random.randint(1, 100) if i != 0 and j !=0  and i != j else 0 for i in V] for j in V]
C = A

# Assume we have 3 compartments in the vehicle, each of depth 800.
nb_compartments = 3
M = [i for i in range(nb_compartments)]
Capacities = [800 for i in range(nb_compartments)]

# Demands (5 items)
Demands = [0] + [400] * nb_items + [-400] * nb_items


In [4]:
print(f"Number of requests {nb_items}")
print(f"The cost matrix \n{C}")

Number of requests 5
The cost matrix 
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 50, 98, 54, 6, 34, 66, 63, 52, 39], [0, 62, 0, 46, 75, 28, 65, 18, 37, 18, 97], [0, 13, 80, 0, 33, 69, 91, 78, 19, 40, 13], [0, 94, 10, 88, 0, 43, 61, 72, 13, 46, 56], [0, 41, 79, 82, 27, 0, 71, 62, 57, 67, 34], [0, 8, 71, 2, 12, 93, 0, 52, 91, 86, 81], [0, 1, 79, 64, 43, 32, 94, 0, 42, 91, 9], [0, 25, 73, 29, 31, 19, 70, 58, 0, 12, 11], [0, 41, 66, 63, 14, 39, 71, 38, 91, 0, 16], [0, 71, 43, 70, 27, 78, 71, 76, 37, 57, 0]]


In [5]:
# Declare and initialize the model
model = Model("1-PDPMS")

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-03-22


#### Decision Variables

- $x_{i,\; j}$
    - $x_{i,\; j} = 1$ if the vehicle is travelling from $i$ to $j$
    - $x_{i,\; j} = 0$ otherwise
- $y_{i,\; k}$
    - $y_{i,\; k} = 1$ if the demand at pickup vertex $i$ is loaded in stack $k$
    - $y_{i,\; k} = 0$ otherwise
- $u_{i}$ is the position of vertex $i$ in the route (order).
    - $0 \leq u_{i} \leq 2n$
    - $u_{0} = 0$ we start at the depot
- $s_{i,\; k}$ is the load of stack $k$ upon **leaving** vertex $i$
    - Domain $ : \{0, 1, .., q_{k}\}$ 

In [6]:
x = model.addVars(V, V, vtype=GRB.BINARY, name="x")
y = model.addVars(V, M, vtype=GRB.BINARY, name="y")
u = model.addVars(V, vtype=GRB.INTEGER, name="u")
s = model.addVars(V, M, vtype=GRB.INTEGER, name="s")

In [7]:
# Set the bounds of the integer vars
for i in u:
    u[i].setAttr(GRB.Attr.LB, 0)    
    u[i].setAttr(GRB.Attr.UB, len(V) - 1)

for i, k in s:
    s[i, k].setAttr(GRB.Attr.LB, 0)    
    s[i, k].setAttr(GRB.Attr.UB, Capacities[k])
    
model.update()

### Constraints

#### Constraint
- Every customer (vertex) is visited once.
    <br>
    <center> $\sum_{j \in V} x_{i, \; j} = 1$    $\; \forall i \in V$ (2)</center>
    <center> $\sum_{j \in V} x_{j, \; i} = 1$    $\; \forall i \in V$ (3)</center>
    <br>
    

In [8]:
# 2
# Every vertex is left once
model.addConstrs((sum(x[i, j] for j in V) == 1 for i in V), name="out_vertex_constr")

# 3
# Every vertex is entered once
model.addConstrs((sum(x[j, i] for j in V) == 1 for i in V), name="in_vertex_constr")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>}

#### Constraint
- The demand of every vertex is assigned to exactly one compartment. (Demand must be fulfilled)
    <br>
    <center> $\sum_{k \in M} y_{i, \; k} = 1$    $\; \forall i \in P$ (4)</center>
    <br>

In [9]:
model.addConstrs((sum(y[i, k] for k in M) == 1 for i in P), name="fulfill_demands_constr")

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

#### Constraint

- If $j$ is visited directly after $i$ i.e. $x_{i, \; j} = 1$, then $u_{j} \geq u_{i} + 1$
    <br>
    <center> $u_{j} \geq u_{i} + 1 - 2n(1 - x_{i, \; j}) \; \forall i \in V, \; \forall j \in P \cup D$ (5)</center>
    <br>

In [10]:
model.addConstrs((u[j] >= u[i] + 1 - (2 * N * (1 - x[i, j])) for i in V for j in PUD), name="visiting_order_constr" )

{(0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>

#### Constraint

- The item must be picked before it's delivered. Precedence constraint between pickups and deliveries.
    <br>
    <center> $u_{n + i} \geq u_{i} + 1 \; \forall i \in P$ (6)</center>
    <br>

In [11]:
model.addConstrs((u[N + i] >= u[i] + 1 for i in P), name="pickup_before_delivery_constr")

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

#### Constraint

- The state of the stack after loading and unloading.
    - The state of the assigned stack after picking is upperbounded by the stack before picking and the picked (loaded) demand.
    
    <br>
    <center> $s_{j, \; k} \geq s_{i, \; k} + d_{j} * y_{j, \; k} - q_{k} * (1 - x_{i, \; j}) \; \forall i \in V, \; \forall j \in P, \; \forall k \in M$ (7)</center>
    <br>
    
    - The state of the assinged stack after delivering is upperbounded by the stack before delivering and the (negative) delivered (unloaded) demand.
    
    <br>
    <center> $s_{(n + j), \; k} \geq s_{i, \; k} + d_{(n + j)} * y_{j, \; k} - q_{k} * (1 - x_{i, \; (n + j)}) \; \forall i \in V, \; \forall j \in P, \; \forall k \in M$ (8)</center>
    <br>
    
    - LIFO stacking constraint. The state of the stack after unloading is bounded by the state of the stack just after loading and the loaded / unloaded demand. (The state after unload $d{j}$ = state after loading $d{j} - d{j}$ equivalent to $ + d_{j + n}$.
    
    <br>
    <center> $s_{(n + j), \; k} \geq s_{j, \; k} + d_{(n + j)} * y_{j, \; k} - q_{k} * (1 - y_{j, \; k}) \; \forall j \in P, \; \forall k \in M$ (9)</center>
    <br>

In [12]:
# 7
model.addConstrs((s[j, k] >= s[i, k] + Demands[j] * y[j, k] - Capacities[k] * (1 - x[i, j]) for i in V for j in P for k in M), name="stack_after_pickup_constr")

# 8
model.addConstrs((s[N + j, k] >= s[i, k] + Demands[N + j] * y[j, k] - Capacities[k] * (1 - x[i, N + j]) for i in V for j in P for k in M), name="stack_after_delivery_constr")

# 9
model.addConstrs((s[N + j, k] >= s[j, k] + Demands[N + j] * y[j, k] - Capacities[k] * (1 - y[j, k]) for j in P for k in M), name="stack_LIFO_constr")

{(1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (5, 0): <gurobi.Constr *Awaiting Model Update*>,
 (5, 1): <gurobi.Constr *Awaiting Model Update*>,
 (5, 2): <gurobi.Constr *Awaiting Model Update*>}

#### Constraint

- We always start at the depot.
    <br><center> $u_{0} = 0$ (10)</center><br>

In [13]:
model.addConstr(u[0] == 0, name="start_at_depot_constr")

<gurobi.Constr *Awaiting Model Update*>

#### Constraints

- Vertices are order along the route.
    <br><center> $1 \leq u_{i}  \leq 2n \; \forall i \in P \cup D$ (11)</center><br>

- We always begin with empty stacks at the depot.
    <br><center> $s_{0, \; k} = 0 \; \forall k \in M$ (12)</center><br>

- The stack capacity is always maintained for all stacks. 
    <br><center> $0 \leq s_{i, \; k}  \leq q_{k} \; \forall i \in P \cup D \; \forall k \in M$ (13)</center><br>

In [14]:
# 11
# constraint implicit in the domain of u

# 12
model.addConstrs((s[0, k] == 0 for k in M), name="empty_stacks_at_depot_constr")

# 13
# constraint implicit in the domain of s

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

### Objectice Function

- Minimize the route cost.
 <br>
     <center>$Min \sum_{i \in V} \sum_{j \in V} c_{i, \; j} * x_{i, \; j}$ (1)</center>

In [15]:
# Set the objective function
model.setObjective((sum(C[i][j] * x[i, j] for i in V for j in V)), GRB.MINIMIZE)

In [16]:
model.write('1-PDPMS_1.lp')

In [17]:
# Run the optimization engine (implicitly runs model.update())
model.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 491 rows, 198 columns and 1886 nonzeros
Model fingerprint: 0x274e7098
Variable types: 0 continuous, 198 integer (154 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 8e+02]
  RHS range        [1e+00, 8e+02]
Presolve removed 69 rows and 37 columns
Presolve time: 0.01s
Presolved: 422 rows, 161 columns, 1652 nonzeros
Variable types: 0 continuous, 161 integer (121 binary)
Found heuristic solution: objective 139.0000000

Root relaxation: objective 1.135600e+02, 88 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  113.56000    0   22  139.00000  113.56000  18.3%     -    0s
     0     0  123.00000    0   

In [18]:
def print_solution(optimized_model):
    for var in optimized_model.getVars():
        if abs(var.x) > 1e-6 or var.vtype != GRB.BINARY:
            print("{0}: {1}".format(var.varName, var.x))
    print("Total cost: {0}".format(optimized_model.objVal))
    return None

# display optimal values of decision variables
print_solution(model)  

x[0,5]: 1.0
x[1,6]: 1.0
x[2,7]: 1.0
x[3,8]: 1.0
x[4,2]: 1.0
x[5,4]: 1.0
x[6,3]: 1.0
x[7,1]: 1.0
x[8,9]: 1.0
x[9,10]: 1.0
x[10,0]: 1.0
y[1,2]: 1.0
y[2,2]: 1.0
y[3,2]: 1.0
y[4,2]: 1.0
y[5,1]: 1.0
u[0]: 0.0
u[1]: 5.0
u[2]: 3.0
u[3]: 7.0
u[4]: 2.0
u[5]: 1.0
u[6]: 6.0
u[7]: 4.0
u[8]: 8.0
u[9]: 9.0
u[10]: 10.0
s[0,0]: 0.0
s[0,1]: 0.0
s[0,2]: 0.0
s[1,0]: -0.0
s[1,1]: 400.0
s[1,2]: 800.0
s[2,0]: -0.0
s[2,1]: 400.0
s[2,2]: 800.0
s[3,0]: -0.0
s[3,1]: 400.0
s[3,2]: 800.0
s[4,0]: -0.0
s[4,1]: 400.0
s[4,2]: 400.0
s[5,0]: -0.0
s[5,1]: 400.0
s[5,2]: -0.0
s[6,0]: -0.0
s[6,1]: 400.0
s[6,2]: 400.0
s[7,0]: -0.0
s[7,1]: 400.0
s[7,2]: 400.0
s[8,0]: -0.0
s[8,1]: 400.0
s[8,2]: 400.0
s[9,0]: -0.0
s[9,1]: 400.0
s[9,2]: -0.0
s[10,0]: -0.0
s[10,1]: -0.0
s[10,2]: -0.0
Total cost: 139.0


In [19]:
# The order of visiting the vertices
nexts = [j for i in V for j in V if x[i, j].x == 1]

# We always start at the depot
order = [0] 
curr = nexts[0]

# We always end at the depot
while (curr !=0):
    order.append(curr)
    curr = nexts[curr]
    
print(f"The order of visiting the vertices is {order[1:]}\n")
for i in order[1:]:
    action = "pickup" if i in P else "deliver"
    prop = "to" if i in P else "from"
    pick_idx = i if i in P else i - N
    comp = next(k for k in M if abs(y[pick_idx, k].x) > 1e-6)
    print(f"@ Vertex: {i} {action} item {pick_idx} {prop} {comp}")

The order of visiting the vertices is [5, 4, 2, 7, 1, 6, 3, 8, 9, 10]

@ Vertex: 5 pickup item 5 to 1
@ Vertex: 4 pickup item 4 to 2
@ Vertex: 2 pickup item 2 to 2
@ Vertex: 7 deliver item 2 from 2
@ Vertex: 1 pickup item 1 to 2
@ Vertex: 6 deliver item 1 from 2
@ Vertex: 3 pickup item 3 to 2
@ Vertex: 8 deliver item 3 from 2
@ Vertex: 9 deliver item 4 from 2
@ Vertex: 10 deliver item 5 from 1
